In [1]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd

In [2]:
# create engine
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')
engine.table_names()

['Bookings', 'Facilities', 'Members']

In [3]:
def run_query(engine, q):
    with engine.connect() as con:
        rs = con.execute(q)
        df = pd.DataFrame(rs.fetchmany(size=5))
        df.columns = rs.keys()
    return df

In [4]:
# Q10: Produce a list of facilities with a total revenue less than 1000.
# The output of facility name and total revenue, sorted by revenue. Remember
# that there's a different cost for guests and members!
q = """ 
    SELECT 
     F.name
    ,SUM(CASE WHEN M.memid = 0 THEN F.guestcost ELSE F.membercost END * B.slots ) AS revenue
    FROM Bookings B
    INNER JOIN Facilities F ON B.facid = F.facid
    LEFT JOIN Members M ON B.memid = M.memid
    GROUP BY 
    F.name
    HAVING(revenue) < 1000
    ORDER BY 
    revenue
    """ 
run_query(engine, q)

,name,revenue
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270


In [5]:
# Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order
q = """ 
    SELECT 
    M.memid
    ,M.firstname
    ,M.surname
    ,M.address
    ,M.zipcode
    ,M.telephone
    ,R.firstname || ' ' || R.surname as recommendedby_name
    FROM Members M
    INNER JOIN Members R ON M.recommendedby = R.memid
    WHERE M.recommendedby > 0
    ORDER BY
    M.surname
    ,M.firstname
    """ 
run_query(engine, q)

,memid,firstname,surname,address,zipcode,telephone,recommendedby_name
0,15,Florence,Bader,"264 Ursula Drive, Westford",84923,(833) 499-3527,Ponder Stibbons
1,12,Anne,Baker,"55 Powdery Street, Boston",80743,844-076-5141,Ponder Stibbons
2,16,Timothy,Baker,"329 James Street, Reading",58393,833-941-0824,Jemima Farrell
3,8,Tim,Boothe,"3 Bloomsbury Close, Reading, 00234",234,(811) 433-2547,Tim Rownam
4,5,Gerald,Butters,"1065 Huntingdon Avenue, Boston",56754,(844) 078-4130,Darren Smith


In [6]:
# Q12: Find the facilities with their usage by member, but not guests
q = """ 
    SELECT 
    F.name
    ,M.memid
    ,M.firstname || ' ' || M.surname as member_name
    ,SUM(B.slots) AS usage
    FROM Bookings B
    INNER JOIN Facilities F ON B.facid = F.facid
    INNER JOIN Members M ON B.memid = M.memid
    WHERE M.memid > 0
    GROUP BY
    F.name
    ,M.memid
    ,member_name
    ORDER BY
    usage DESC
    """ 
run_query(engine, q)

,name,memid,member_name,usage
0,Badminton Court,1,Darren Smith,432
1,Pool Table,3,Tim Rownam,282
2,Massage Room 1,3,Tim Rownam,176
3,Tennis Court 1,5,Gerald Butters,171
4,Tennis Court 2,8,Tim Boothe,168


In [7]:
# Q13: Find the facilities usage by month, but not guests
q = """ 
    SELECT 
    F.name
    ,strftime('%Y-%m', B.starttime) as month
    ,SUM(B.slots) AS usage
    FROM Bookings B
    INNER JOIN Facilities F ON B.facid = F.facid
    INNER JOIN Members M ON B.memid = M.memid
    WHERE M.memid > 0
    GROUP BY
    F.name
    ,month
    ORDER BY
    usage DESC
    """ 
run_query(engine, q)

,name,month,usage
0,Badminton Court,2012-09,507
1,Pool Table,2012-09,443
2,Tennis Court 1,2012-09,417
3,Badminton Court,2012-08,414
4,Tennis Court 2,2012-09,414
